Declaration A

Start by importing bbSearch to perform search algorithms

In [ ]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py

from bbmodcache.bbSearch import SearchProblem, search

Code from Search exercise 7. Creates the grids with their respective blocks and their respective colours

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy

plt.ioff()  ## Turn off immediate display of plots

COLORS = ["lightgray", "red", "blue", "green", "yellow",
          "orange", "purple", "pink", "brown"]

class BlockState:

      # Specify mapping from directions to grid coordinate offsets:
      neighbour_offset = {"left": (0,-1), "right": (0,+1), "down":(+1,0), "up":(-1,0)}

      def __init__( self, blockstate, colors=COLORS ):
        self.blockstate = blockstate
        self.nrows = len(blockstate)
        self.ncols = len(blockstate[0])
        self.blocknums = set().union(*[set(row) for row in blockstate])
        self.blocknums = self.blocknums - {0}
        self.blocknumlist = list(self.blocknums)
        self.colors = colors

      def __repr__(self):
        return( str( self.blockstate ))

      # Find the cells occupied by a given number
      def blockcells( self, blocknum ):
          blockcells = []
          for row in range(self.nrows):
            for col in range(self.ncols):
              if self.blockstate[row][col] == blocknum:
                blockcells.append((row,col))
          return blockcells

      # Test if a cell is free (unblocked) in a given direction
      # Free if not blocked by edge of grid or by a cell of different colour
      def free_cell( self, direction, cell ):
        row, col = cell
        offrow, offcol = BlockState.neighbour_offset[direction]
        neighrow, neighcol = (row + offrow, col + offcol)
        if not (0 <= neighrow < self.nrows): return False #at top or bottom
        if not (0 <= neighcol < self.ncols): return False #at left or right
        neighval = self.blockstate[neighrow][neighcol]
        # Neighboring cell must be empty or part of the same coloured block
        return  (neighval==0 or neighval==self.blockstate[row][col])

      def free_block( self, direction, blockn ):
          blockcells = self.blockcells(blockn)
          for cell in blockcells:
            if not self.free_cell(direction, cell):
              return False
          return True

      def possible_moves(self):
        moves = []
        for blocknum in self.blocknumlist:
          for direction in ["left", "right", "down", "up"]:
              if self.free_block(direction, blocknum):
                  moves.append((blocknum, direction))
        return moves

      def next_state(self, move):
          next_blockstate = deepcopy(self.blockstate)
          blockno, direction = move
          cells = self.blockcells(blockno)
          ## first clear all cells of the block (set to 0)
          for cell in cells:
            row, col = cell
            next_blockstate[row][col] = 0
          rowoff, coloff = BlockState.neighbour_offset[direction]
          ## now set all neighbour cells (in move direction) to be
          ## cells with the blocknumber
          for cell in cells:
            row, col = cell
            next_blockstate[row+rowoff][col+coloff] = blockno
          return BlockState(next_blockstate)

      def color_key(self):
          return {b:self.colors[b] for b in self.blocknumlist}

      def figure(self, scale=0.5):
          nrows = self.nrows
          ncols = self.ncols
          fig, ax = plt.subplots(figsize=(ncols*scale+0.1,nrows*scale+0.1))
          plt.close(fig)
          ax.set_axis_off() # Don't show border lines and coordinate values

          frame = patches.Rectangle((0,0),1,1, linewidth=5, edgecolor='k', facecolor='w')
          ax.add_patch(frame)

          for row in range(nrows):
            for col in range(ncols):
                greyrect = patches.Rectangle( (((col*0.9)/ncols)+0.05,
                                               (((nrows-row-1)*0.9)/nrows)+0.05 ),
                                            0.9/ncols, 0.9/nrows,
                                            linewidth=1, edgecolor="gray", facecolor="lightgray")
                ax.add_patch(greyrect)

          for row in range(nrows):
            for col in range(ncols):
                cellval = self.blockstate[row][col]
                if cellval > 0:
                  cellcol = COLORS[cellval]
                  rect = patches.Rectangle( (((col*0.9)/ncols)+0.05,
                                             (((nrows-row-1)*0.9)/nrows)+0.05 ),
                                            0.9/ncols, 0.9/nrows,
                                            linewidth=0, edgecolor=cellcol, facecolor=cellcol)
                  ax.add_patch(rect)
          return fig

      def display(self):
          display(self.figure())

In [ ]:
from copy import deepcopy
class SlidingBlocksPuzzle( SearchProblem ):

    def __init__( self, initial_state, goal, colors=COLORS ):
        """
        The __init__ method must set the initial state for the search.
        Arguments could be added to __init__ and used to configure the
        initial state and/or other aspects of a problem instance.
        """
        self.initial_state = BlockState(initial_state, colors=colors)
        self.colors = colors
        self.goal = BlockState(goal)

    def info(self):
        print("Solve the following sliding blocks problem.")
        print("Get from this initial state:")
        self.initial_state.display()
        print("To a state incorporating the following block position(s):")
        self.goal.display()
        print("You need to slide the red block to cover the bottom right square.")

    def possible_actions(self, state):
        return state.possible_moves()

    def successor(self, state, action):
        """
        This takes a state and an action and returns the new state resulting
        from doing that action in that state. You can assume that the given
        action is in the list of 'possible_actions' for that state.
        """
        return state.next_state(action)

    def goal_test(self, state):
        """
        For the sliding blocks puzzles, the goal condition is reached when
        all block possitions specified in the given goal state are satisfied by
        the current state. But empty positions (ie 0s) in the goal are ignored,
        so can be occupied by blocks in the current sate.
        """
        for row in range(state.nrows):
          for col in range(state.ncols):
            goalnum = self.goal.blockstate[row][col]
            if goalnum==0:
              continue
            if goalnum != state.blockstate[row][col]:
              return False
        return True


    def cost(self, path, state):
        """
        This is an optional method that you only need to define if you are using
        a cost based algorithm such as "uniform cost" or "A*". It should return
        the cost of reaching a given state via a given path.
        If this is not re-defined, it will is assumed that each action costs one unit
        of effort to perform, so it returns the length of the path.
        """
        return len(path)

    def display_action(self, action):
        """
        You can set the way an action will be displayed in outputs.
        """
        print((self.colors[action[0]], action[1]))

    def display_state(self, state):
        """
        You can set the way a state will be displayed in outputs.
        """
        state.display()

    def display_state_path( self, actions ):
        """
        This defines output of a solution path when a list of actions
        is applied to the initial state. It assumes it is a valid path
        with all actions being possible in the preceeding state.
        You probably don't need to override this.
        """
        s = self.initial_state
        self.display_state(s)
        for a in actions:
            self.display_action(a)
            s = self.successor(s,a)
            self.display_state(s)

After some experimentation we decided to investigate the following cases of Sliding Block Puzzle. We chose a 4x4 grid for the smaller grid as well as an 8x7 grid for the second, larger grid. We placed the same blocks on both grids and the goal for both is to move the red L chaped block to the bottom left of both grid. 

In [ ]:
small_initial = [[1,0,0,2],
         [1,1,0,0],
         [0,0,0,4],
         [3,3,0,4]]

small_goal = [[0,0,0,0],
         [0,0,0,0],
         [0,0,1,0],
         [0,0,1,1]]

large_initial = [[1,0,0,0,0,0,0,2],
         [1,1,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,4],
         [3,3,0,0,0,0,0,4]]


large_goal = [[0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,1,0],
         [0,0,0,0,0,0,1,1]]

small_puzzle =  SlidingBlocksPuzzle( small_initial,  small_goal)
large_puzzle =  SlidingBlocksPuzzle( large_initial,  large_goal)

The first test is to show how the size of the grids may effect the time it takes to complete the search as well as how the different search algorithms effect the time and path length. Therefore, for every test, loop check is true and the node limit is 5000

In [ ]:
small_search = search( small_puzzle, 'BF/FIFO', 5000, loop_check=True, randomise=False, return_info=True)
large_search = search( large_puzzle, 'BF/FIFO', 5000, loop_check=True, randomise=False, return_info=True)

This test is for Depth first search

In [ ]:
small_Df = search( small_puzzle, 'DF/LIFO', 5000, loop_check=True, randomise=False, return_info=True)
large_DF = search( large_puzzle, 'DF/LIFO', 5000, loop_check=True, randomise=False, return_info=True)

This is the same as the depth first test but randomise is true

In [ ]:
small_DFR = search( small_puzzle, 'DF/LIFO', 5000, loop_check=True, randomise=True, return_info=True)
large_DFR = search( large_puzzle, 'DF/LIFO', 5000, loop_check=True, randomise=True, return_info=True)

We have no introduced the cost aspect of the search. This cost function is from Search Exercise 6

In [ ]:
def cost(path, state):
        return len(path)

small_cost = search(small_puzzle, 'BF/FIFO', 5000, cost=cost, loop_check=True, return_info=True)
large_cost = search(large_puzzle, 'BF/FIFO', 5000, cost=cost, loop_check=True, return_info=True)

This A* algorithm uses the cost function from before and now uses heuristics. This heuristic is from Search Exercise 7

In [ ]:
def red_right_heuristic(state):
    for row in state.blockstate:
      for i, col in enumerate(row):
          if col == 1:
            return 6-i

small_A = search( small_puzzle, 'DF/LIFO', 5000, loop_check=True, randomise=True, return_info=True, cost=cost, heuristic=red_right_heuristic)
large_A = search( large_puzzle, 'DF/LIFO', 5000, loop_check=True, randomise=True, return_info=True, cost=cost, heuristic=red_right_heuristic)


This table shows a summary of every test that was just performed and it is from Search Exercise 7. From the results, all the large tests failed with the low node limit of 5000.

In [ ]:
TEST_RESULTS =[small_search, large_search, small_Df, large_DF, small_DFR, large_DFR,small_A, large_A]

# Specify symbols for termination conditions:
short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("Test    #max   Result   #gen     #inQ    Time s")
for i, test in enumerate(TEST_RESULTS):
    max  = test['args']['max_nodes']
    tc  = test['result']['termination_condition']
    stc = short_tc[tc]

    ng  = test['search_stats']['nodes_generated']
    nq  = test['search_stats']['nodes_left_in_queue']
    time = round( test['search_stats']['time_taken'], 2 )
    print( f"{i:>3}: {max:>8}    {stc}  {ng:>8} {nq:>8}     {time} ")

In [ ]:
small_search2 = search( small_puzzle, 'BF/FIFO', 30000, return_info=True, loop_check=True, randomise=False)
large_search2 = search( large_puzzle, 'BF/FIFO', 30000, return_info=True, loop_check=True, randomise=False)

In [ ]:
small_Df2 = search( small_puzzle, 'DF/LIFO', 30000, return_info=True, loop_check=True, randomise=False)
large_DF2 = search( large_puzzle, 'DF/LIFO', 30000, return_info=True, loop_check=True, randomise=False)

In [ ]:
small_DFR2 = search( small_puzzle, 'DF/LIFO', 30000, return_info=True, loop_check=True, randomise=True)
large_DFR2 = search( large_puzzle, 'DF/LIFO', 30000, return_info=True, loop_check=True, randomise=True)

In [ ]:
small_cost2 = search(small_puzzle, 'BF/FIFO', 30000, return_info=True, cost=cost, loop_check=True)
large_cost2 = search(large_puzzle, 'BF/FIFO', 30000, return_info=True, cost=cost, loop_check=True)

In [ ]:
small_A2 = search( small_puzzle, 'DF/LIFO', 30000, loop_check=True, randomise=True, return_info=True, cost=cost, heuristic=red_right_heuristic)
large_A2 = search( large_puzzle, 'DF/LIFO', 30000, loop_check=True, randomise=True, return_info=True, cost=cost, heuristic=red_right_heuristic)

In [ ]:
TEST_RESULTS =[small_search2, large_search2, small_Df2, large_DF2, small_DFR2, large_DFR2, small_A2, large_A2]

# Specify symbols for termination conditions:
short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("Test    #max   Result   #gen     #inQ    Time s")
for i, test in enumerate(TEST_RESULTS):
    max  = test['args']['max_nodes']
    tc  = test['result']['termination_condition']
    stc = short_tc[tc]

    ng  = test['search_stats']['nodes_generated']
    nq  = test['search_stats']['nodes_left_in_queue']
    time = round( test['search_stats']['time_taken'], 2 )
    print( f"{i:>3}: {max:>8}    {stc}  {ng:>8} {nq:>8}     {time} ")